# COMP4124 Lab 02: Introduction to Spark DataFrames

In this lab, we will start to work with Spark DataFrames. If you have worked with pandas DataFrames before, you are likely to find working with this kind of tabular data structure familiar.

Although Spark DataFrames offer similar operations to pandas DataFrames, the API itself is quite different to work with. Additionally, it is important to remember that Spark DataFrames are *distributed*, while pandas DataFrames are within a single machine. This has implications for *how* we work with the data, and in future weeks we will cover more of the technical details of Spark in order to explain this. For this lab, we don't go into much depth on how Spark works. Instead, we focus first on a preliminary introduction to working with the DataFrame API.

Throughout this lab, you may want to refer to the [PySpark Docs](https://spark.apache.org/docs/latest/api/python/reference/index.html).

## Dataset

The dataset we will be using contains information on Open and Green Spaces in Nottingham. The original dataset is from the [Nottingham City Council Open Data Portal](https://geoportal-nottmcitycouncil.opendata.arcgis.com/). A (partially) cleaned version of the dataset is availabe on Moodle under the Lab 2 folder - **this cleaned version is what should be used for this lab**.

> Nottingham City Council. (2024) 'Open and Green Spaces'. Available at: https://geoportal-nottmcitycouncil.opendata.arcgis.com/datasets/c3e46990f825426789e46d966ab5434d_70/about . (Accessed: 03/02/2024).

The link above contains more detail on the attributes of this dataset, as well as a link to the [data definition document](https://nottmcitycouncil.maps.arcgis.com/sharing/rest/content/items/c35ec1017beb414c82f8ec509149b5a6/data).


## Set-Up PySpark within Google Colab

Before we can use PySpark in a Google Colab notebook, we first need to install PySpark and start a Spark Session.

If you are running this on your own device where you have PySpark installed, **delete the cell `!pip install pyspark`**.

In [1]:
!pip install pyspark

**Initialise the `SparkSession`:**

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Lab02") \
    .getOrCreate()

sc = spark.sparkContext

**Import built-in DataFrame operations:** PySpark has numerous built-in functions for operations with DataFrames. We need to explicitly import these before we can use them. I give this the alias `F` here - this avoids ambiguity that could arise between PySpark functions and Python functions that have the same name.

In [3]:
import pyspark.sql.functions as F

## Create a DataFrame by reading from a csv file

If you are running this in Colab, remember that you will need to make the csv file available to the runtime. Refer to Lab 1 if you have forgotten how to do this.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/COMP 4124')  # Replace with your actual folder path

In [9]:
df_spaces = spark.read.csv('/content/drive/MyDrive/COMP 4124/open_spaces_cleaned.csv', header=True)

We can use `DataFrame.show(n)` to print the first `n` rows of the DataFrame.

In [10]:
df_spaces.show(10)

+-------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------+--------------------+
|Site_ID|                Name|    Hierarchy|    Primary_Typology|  Secondary_Typology|        Specific_Use|         Site_Access|          Site_Owner|         Whole_Part|Site_Area_Ha|                Ward|
+-------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------+--------------------+
|     62| Victoria Embankment|  Destination|   Parks and Gardens|Provision for Chi...|       Paddling Pool|Freely accessible...|Nottingham City C...|Part of larger site|      0.2454|             Meadows|
|   2401|Elliott Durham Sc...|         NULL|Outdoor Sports Fa...|                NULL|            Football|    No public access|                NULL|Part of larger site|       1.995|  

If you don't want values to be truncated, set the `truncate` parameter to `False`.

In [ ]:
df_spaces.show(10,truncate=False)

+-------+------------------------------------------+-------------+-----------------------+---------------------------------------+--------------------+------------------------------------------+-----------------------+-------------------+------------+----------------------+
|Site_ID|Name                                      |Hierarchy    |Primary_Typology       |Secondary_Typology                     |Specific_Use        |Site_Access                               |Site_Owner             |Whole_Part         |Site_Area_Ha|Ward                  |
+-------+------------------------------------------+-------------+-----------------------+---------------------------------------+--------------------+------------------------------------------+-----------------------+-------------------+------------+----------------------+
|62     |Victoria Embankment                       |Destination  |Parks and Gardens      |Provision for Children and Young People|Paddling Pool       |Freely accessible to pub

Let's have a look at the *schema* of our DataFrame.

In [ ]:
df_spaces.printSchema()

root
 |-- Site_ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Hierarchy: string (nullable = true)
 |-- Primary_Typology: string (nullable = true)
 |-- Secondary_Typology: string (nullable = true)
 |-- Specific_Use: string (nullable = true)
 |-- Site_Access: string (nullable = true)
 |-- Site_Owner: string (nullable = true)
 |-- Whole_Part: string (nullable = true)
 |-- Site_Area_Ha: string (nullable = true)
 |-- Ward: string (nullable = true)



All of our attributes are strings! This is because Spark by default does not infer the schema from the data when reading from a csv file. If we look at the data, we can see that we'd probably expect `Site_ID` to be an integer and `Site_Area_Ha` to be a float.

We can cast these columns to the type we want.

In [ ]:
df_spaces \
  .withColumn('Site_ID',F.col('Site_ID').cast('integer')) \
  .withColumn('Site_Area_Ha',F.col('Site_Area_Ha').cast('float'))

DataFrame[Site_ID: int, Name: string, Hierarchy: string, Primary_Typology: string, Secondary_Typology: string, Specific_Use: string, Site_Access: string, Site_Owner: string, Whole_Part: string, Site_Area_Ha: float, Ward: string]

Let's check the schema again.

In [ ]:
df_spaces.printSchema()

root
 |-- Site_ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Hierarchy: string (nullable = true)
 |-- Primary_Typology: string (nullable = true)
 |-- Secondary_Typology: string (nullable = true)
 |-- Specific_Use: string (nullable = true)
 |-- Site_Access: string (nullable = true)
 |-- Site_Owner: string (nullable = true)
 |-- Whole_Part: string (nullable = true)
 |-- Site_Area_Ha: string (nullable = true)
 |-- Ward: string (nullable = true)



Nothing has changed - all our attributes are still strings. This is because DataFrames are *immutable* - they cannot be changed. Operations on DataFrames create new DataFrames, rather than making changes to the original DataFrame. If we want to be able to refer to our changes, we will need to assign the result of the operations to a variable.

In [ ]:
df_spaces = df_spaces \
  .withColumn('Site_ID',F.col('Site_ID').cast('integer')) \
  .withColumn('Site_Area_Ha',F.col('Site_Area_Ha').cast('float'))

In [ ]:
df_spaces.printSchema()

root
 |-- Site_ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Hierarchy: string (nullable = true)
 |-- Primary_Typology: string (nullable = true)
 |-- Secondary_Typology: string (nullable = true)
 |-- Specific_Use: string (nullable = true)
 |-- Site_Access: string (nullable = true)
 |-- Site_Owner: string (nullable = true)
 |-- Whole_Part: string (nullable = true)
 |-- Site_Area_Ha: float (nullable = true)
 |-- Ward: string (nullable = true)



This now looks like a reasonable schema!

## Basic DataFrame manipulation

Let's have a look at the type of `df_spaces`.

In [ ]:
type(df_spaces)

pyspark.sql.dataframe.DataFrame

Let's see what happens when we count the number of rows in the DataFrame.

In [ ]:
num_rows = df_spaces.count()
print(f'There are {num_rows} rows in df_spaces. The type of num_rows is {type(num_rows)}')

There are 2449 rows in df_spaces. The type of num_rows is <class 'int'>


`df_spaces` is a [PySpark DataFrame](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.html): "A distributed collection of data grouped into named columns." This means the DataFrame is parallelised, and operations on it are occurring in parallel across multiple nodes/executors.

When we used the `.count()` function, a  normal Python `int` was returned. The `.count()` operation has happened in parallel, and the results from the different executors have been combined into one value on the *driver node*.

Don't worry too much about the distinction for now - the upcoming lectures and labs will go into more detail on the implications of this!

**Select a subset of the columns and `show()` the resulting DataFrame.** Note that `show()` also returns values to the driver node in order to print it to screen - for this reason, you should not show too many lines of the DataFrame.

In [ ]:
df_spaces.select(['Name','Site_Area_Ha']).show(10)

+--------------------+------------+
|                Name|Site_Area_Ha|
+--------------------+------------+
| Victoria Embankment|      0.2454|
|Elliott Durham Sc...|       1.995|
|Hempshill Lane Re...|      1.9917|
|Birchover Recreat...|      1.9537|
|Clifton Village C...|      1.9018|
|     Bilborough Park|      1.8684|
|Wollaton Sports A...|      1.7815|
|Wollaton Sports A...|      1.7815|
|Highfields Sports...|      1.5583|
|      Stockhill Park|      1.5459|
+--------------------+------------+
only showing top 10 rows



<font color='blue'>**Task:** Check the PySpark docs for how to drop columns. Drop the `Primary_Typology`, `Secondary_Typology`, and `Specific_Use` columns, and then assign the resulting DataFrame back to the `df_spaces` variable. Then `show()` the DataFrame.</font>

In [ ]:
df_spaces = df_spaces.drop('Primary_Typology', 'Secondary_Typology', 'Specific_Use')

In [ ]:
df_spaces.show()

+-------+--------------------+-------------+--------------------+--------------------+-------------------+------------+--------------------+
|Site_ID|                Name|    Hierarchy|         Site_Access|          Site_Owner|         Whole_Part|Site_Area_Ha|                Ward|
+-------+--------------------+-------------+--------------------+--------------------+-------------------+------------+--------------------+
|     62| Victoria Embankment|  Destination|Freely accessible...|Nottingham City C...|Part of larger site|      0.2454|             Meadows|
|   2401|Elliott Durham Sc...|         NULL|    No public access|                NULL|Part of larger site|       1.995|           Mapperley|
|     93|Hempshill Lane Re...|Neighbourhood|Freely accessible...|                NULL|Part of larger site|      1.9917|             Bulwell|
|    266|Birchover Recreat...|Neighbourhood|Freely accessible...|                NULL|Part of larger site|      1.9537|          Bilborough|
|   2391|Clif

<font color='blue'>**Task:** Find out how to rename a column. Rename the `Name` column to `Site_Name`. Don't forget to reassign the resulting DataFrame back to `df_spaces` so the change is maintained - use `.show()` to check that this has occurred.</font>

In [ ]:
df_spaces = df_spaces.withColumnRenamed('Name','Site_Name')
df_spaces.show(10)

+-------+--------------------+-------------+--------------------+--------------------+-------------------+------------+--------------------+
|Site_ID|           Site_Name|    Hierarchy|         Site_Access|          Site_Owner|         Whole_Part|Site_Area_Ha|                Ward|
+-------+--------------------+-------------+--------------------+--------------------+-------------------+------------+--------------------+
|     62| Victoria Embankment|  Destination|Freely accessible...|Nottingham City C...|Part of larger site|      0.2454|             Meadows|
|   2401|Elliott Durham Sc...|         NULL|    No public access|                NULL|Part of larger site|       1.995|           Mapperley|
|     93|Hempshill Lane Re...|Neighbourhood|Freely accessible...|                NULL|Part of larger site|      1.9917|             Bulwell|
|    266|Birchover Recreat...|Neighbourhood|Freely accessible...|                NULL|Part of larger site|      1.9537|          Bilborough|
|   2391|Clif

**Generally, there are many different ways to achieve the same result with PySpark.** For example, the following lines of code all filter the DataFrame, keeping only the rows where the site area is greater than 5 hectares, before counting the resulting rows.

In [ ]:
print('1:',df_spaces.filter(df_spaces.Site_Area_Ha > 5.0).count())
print('2:',df_spaces.where(df_spaces.Site_Area_Ha > 5.0).count()) # using `where` instead of `filter`
print('3:',df_spaces.filter(F.col('Site_Area_Ha') > 5.0).count()) # using `F.col()` to get the column instead of dot notation
print('4:',df_spaces.filter(df_spaces['Site_Area_Ha'] > 5.0).count()) # using [] to get the column instead of dot notation
print('5:',df_spaces.filter('Site_Area_Ha > 5.0').count()) # using an SQL-like expression as a string

1: 93
2: 93
3: 93
4: 93
5: 93


**Filter depending on a list using `isin()`.**

In [ ]:
wards = ['Wollaton West','Lenton & Wollaton East']
df_spaces.filter(df_spaces.Ward.isin(wards)).select('Site_Name','Ward').show(truncate=False)

+------------------------------------------+----------------------+
|Site_Name                                 |Ward                  |
+------------------------------------------+----------------------+
|Wollaton Sports Association Cricket Pitch |Wollaton West         |
|Wollaton Sports Association Football Pitch|Wollaton West         |
|Highfields Sports Club                    |Lenton & Wollaton East|
|Grove Farm                                |Lenton & Wollaton East|
|Highfields Sports Pitches                 |Lenton & Wollaton East|
|Grove Farm                                |Lenton & Wollaton East|
|Nottingham Tennis Centre                  |Lenton & Wollaton East|
|Fernwood School                           |Wollaton West         |
|Dave Eastwood Sports Ground               |Lenton & Wollaton East|
|Wollaton Park                             |Wollaton West         |
|Greenwood Meadows FC                      |Lenton & Wollaton East|
|Highfields Park 1 (NCC Owned)             |Lent

<font color='blue'>**Task:** Display all rows of the DataFrame where `Site_Name` contains the string `'University'`. You will need to find the appropriate function to use inside of `filter()` for the condition.</font>

In [ ]:
#Find any site with the word University
df_spaces.filter(df_spaces.Site_Name.contains('University')).show(truncate=False)

+-------+---------------------------------------------------+---------+---------------------+------------------------+----------+------------+----------------------+
|Site_ID|Site_Name                                          |Hierarchy|Site_Access          |Site_Owner              |Whole_Part|Site_Area_Ha|Ward                  |
+-------+---------------------------------------------------+---------+---------------------+------------------------+----------+------------+----------------------+
|2258   |University of Nottingham - University Park         |         |No access information|                        |Whole Site|64.3309     |Lenton & Wollaton East|
|2256   |University of Nottingham - Highfields Playing Field|Local    |No access information|University of Nottingham|Whole Site|13.144      |Lenton & Wollaton East|
|2257   |University of Nottingham - Jubilee Campus          |         |No access information|                        |Whole Site|10.7413     |Lenton & Wollaton East|
|225

**Let's get the distinct values from the `Site_Access` column.**

In [ ]:
df_spaces.select('Site_Access').distinct()

DataFrame[Site_Access: string]

The `distinct()` function is still returning a DataFrame. If you've had a look at the attribute information for the dataset, you will know there are very few distinct values for the `Site_Access` column. Nonetheless, Spark returns the result as a distributed DataFrame - probably a good thing in case you call it when there are many distinct values!

If we want to view the result, we will need to `show()` the resulting DataFrame.

In [ ]:
df_spaces.select('Site_Access').distinct().show(truncate=False)

+------------------------------------------------+
|Site_Access                                     |
+------------------------------------------------+
|Not Accessible                                  |
|Restricted access: members/tenants only         |
|No public access                                |
|Freely accessible to public: opening hours      |
|Restricted access: other                        |
|No access information                           |
|Freely accessible to public                     |
|Freely accessible to public: unofficial/de facto|
+------------------------------------------------+



If we want to use these distinct values later in our code, we can `.collect()` the DataFrame. This brings **the entire DataFrame** back to the driver node. You should only use `.collect()` in your code if you are very sure that the DataFrame is small enough to fit in memory of your driver node. If it is too large, you are likely to get an out of memory error.

The DataFrame we obtained via `distinct()` is definitely small enough, so we can just `.collect()` it.

In [ ]:
df_spaces.select('Site_Access').distinct().collect()

[Row(Site_Access='Not Accessible'),
 Row(Site_Access='Restricted access: members/tenants only'),
 Row(Site_Access='No public access'),
 Row(Site_Access='Freely accessible to public: opening hours'),
 Row(Site_Access='Restricted access: other'),
 Row(Site_Access='No access information'),
 Row(Site_Access='Freely accessible to public'),
 Row(Site_Access='Freely accessible to public: unofficial/de facto')]

Look at the output above - DataFrames are collected back to the driver as a list of Rows. It is easy to change this into, for example, a list of strings that we can use somewhere else in our program.

In [ ]:
access_values = [row.Site_Access for row in df_spaces.select('Site_Access').distinct().collect()]
access_values

['Not Accessible',
 'Restricted access: members/tenants only',
 'No public access',
 'Freely accessible to public: opening hours',
 'Restricted access: other',
 'No access information',
 'Freely accessible to public',
 'Freely accessible to public: unofficial/de facto']

In the below task we are going to be retrieving rows for destinations that are freely accessible to the public. So that we don't have to hardcode the list of freely accessible values, we can just create it from `access_values` using standard list comprehension.

In [ ]:
free_access = [s for s in access_values if 'Freely accessible' in s]
free_access

['Freely accessible to public: opening hours',
 'Freely accessible to public',
 'Freely accessible to public: unofficial/de facto']

<font color='blue'>**Task:** </font>

1.   <font color='blue'>Get the distinct values from the `Hierarchy` column and display them as a list. You will see that we have some missing information in this column - but we will ignore this for now.</font>
2.   <font color='blue'>Retrieve the rows of the DataFrame for spaces which are destinations and are freely accessible (you can use the `free_access` list created above). You will need to find out how to filter based on multiple conditions - there are a few ways this can be done. Show the resulting DataFrame.</font>


In [ ]:
hierarchy_values = [row.Hierarchy for row in df_spaces.select('Hierarchy').distinct().collect()]
hierarchy_values

['Not Accessible',
 'Destination',
 'Private',
 'Not in Use',
 'Local',
 'In Use',
 'Neighbourhood',
 ' ',
 'City',
 None]

In [ ]:
df_spaces.filter((df_spaces.Hierarchy == 'Destination') & (df_spaces.Site_Access.isin(free_access))).show(truncate=False)

+-------+-----------------------------+-----------+------------------------------------------+-----------------------+-------------------+------------+-----------------------+
|Site_ID|Site_Name                    |Hierarchy  |Site_Access                               |Site_Owner             |Whole_Part         |Site_Area_Ha|Ward                   |
+-------+-----------------------------+-----------+------------------------------------------+-----------------------+-------------------+------------+-----------------------+
|62     |Victoria Embankment          |Destination|Freely accessible to public               |Nottingham City Council|Part of larger site|0.2454      |Meadows                |
|60     |Victoria Embankment          |Destination|Freely accessible to public               |NULL                   |Part of larger site|18.5201     |Meadows                |
|131    |Wollaton Park                |Destination|Freely accessible to public: opening hours|NULL                   |Pa

<font color='blue'>**Task:** Find out how to sort a DataFrame. Display the `Site_Name` and `Site_Area_Ha` columns for the **largest 10 spaces**.</font>

In [ ]:
df_spaces.sort("Site_Area_Ha", ascending=False).select('Site_Name','Site_Area_Ha').show(10,truncate=False)

+------------------------------------------+------------+
|Site_Name                                 |Site_Area_Ha|
+------------------------------------------+------------+
|Wollaton Park                             |209.5824    |
|Bulwell Hall Park                         |98.1392     |
|Colwick Country Park                      |96.3874     |
|Agricultural north of Holme Pit           |75.258      |
|University of Nottingham - University Park|64.3309     |
|Bulwell Hall Golf Course                  |51.1601     |
|Colwick Woods LNR                         |50.6465     |
|Broxtowe Country Park                     |46.0058     |
|Stanton Tip                               |39.6618     |
|Victoria Embankment                       |35.3596     |
+------------------------------------------+------------+
only showing top 10 rows



## Exploring the data and working with Columns

We can use `summary()` to get some summary statistics about a DataFrame. Note, `summary()` also returns a DataFrame.

In [ ]:
df_spaces.summary().show()

+-------+------------------+-----------------+---------+--------------------+----------+-------------------+-----------------+-------------+
|summary|           Site_ID|        Site_Name|Hierarchy|         Site_Access|Site_Owner|         Whole_Part|     Site_Area_Ha|         Ward|
+-------+------------------+-----------------+---------+--------------------+----------+-------------------+-----------------+-------------+
|  count|              2449|             2449|     2214|                2449|      2291|               2449|             2449|         2449|
|   mean|1239.6296447529603|             NULL|     NULL|                NULL|      NULL|               NULL|1.030692402914913|         NULL|
| stddev| 713.7680543507662|             NULL|     NULL|                NULL|      NULL|               NULL|6.277495736873263|         NULL|
|    min|                 1|A52 wooded verges|         |Freely accessible...|          |Part of larger site|           5.0E-4|       Aspley|
|    25%|    

This is not very informative for columns containing strings, or for the `Site_ID` column. Let's limit it to only look at the `Site_Area_Ha` column.

In [ ]:
df_spaces.select('Site_Area_Ha').summary().show()

+-------+-----------------+
|summary|     Site_Area_Ha|
+-------+-----------------+
|  count|             2449|
|   mean|1.030692402914913|
| stddev|6.277495736873263|
|    min|           5.0E-4|
|    25%|           0.0134|
|    50%|           0.0444|
|    75%|           0.2537|
|    max|         209.5824|
+-------+-----------------+



A lot of the functions we use in PySpark are *Column transformations*. If you look at the [Functions page](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html) in the docs, you will see that many of the functions take a `Column` (or column name as a string) as their input parameter, and return a `Column`. For example, `lower()` for Columns containing strings, `isnull()` for checking if the value is null, and `sqrt()` for numerical columns.

If we check the type of `Site_Name`, we see that it is a `Column`.

In [ ]:
type(df_spaces.Site_Name)

pyspark.sql.column.Column

Let's try using `lower()` to convert all the strings in `Site_Name` to lowercase.

In [ ]:
F.lower(df_spaces.Site_Name)

Column<'lower(Site_Name)'>

We can see it has output a column, but we want to see the actual result. What happens if we ty to `show()` this?

In [ ]:
F.lower(df_spaces.Site_Name).show()

TypeError: 'Column' object is not callable

We get an error! This is because we have not yet made a DataFrame containing this column. We need to use column transformations inside of functions such as `withColumn()` and `select()` if we want to end up with a DataFrame.

<font color='blue'>**Task:** Run the following two cells. Make sure you understand the difference between `select()` and `withColumn()`.</font>

In [ ]:
df_spaces.select(F.lower(df_spaces.Site_Name)).show(10)

+--------------------+
|    lower(Site_Name)|
+--------------------+
| victoria embankment|
|elliott durham sc...|
|hempshill lane re...|
|birchover recreat...|
|clifton village c...|
|     bilborough park|
|wollaton sports a...|
|wollaton sports a...|
|highfields sports...|
|      stockhill park|
+--------------------+
only showing top 10 rows



In [ ]:
df_spaces.withColumn('name_lower',F.lower(df_spaces.Site_Name)).show(10)

+-------+--------------------+-------------+--------------------+--------------------+-------------------+------------+--------------------+--------------------+
|Site_ID|           Site_Name|    Hierarchy|         Site_Access|          Site_Owner|         Whole_Part|Site_Area_Ha|                Ward|          name_lower|
+-------+--------------------+-------------+--------------------+--------------------+-------------------+------------+--------------------+--------------------+
|     62| Victoria Embankment|  Destination|Freely accessible...|Nottingham City C...|Part of larger site|      0.2454|             Meadows| victoria embankment|
|   2401|Elliott Durham Sc...|         NULL|    No public access|                NULL|Part of larger site|       1.995|           Mapperley|elliott durham sc...|
|     93|Hempshill Lane Re...|Neighbourhood|Freely accessible...|                NULL|Part of larger site|      1.9917|             Bulwell|hempshill lane re...|
|    266|Birchover Recreat..

<font color='blue'>**Task:** Use the [median function](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.median.html) inside a `select()` to compute the median of the `Site_Area_Ha` column. Show the resulting DataFrame. You should see that the resulting column does not have a very user-friendly name.

[Check the docs ](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.alias.html) for the `alias()` function, and use it to name the resulting column `Area_Median` as part of that same `select()`.</font>

<font color='blue'>**Question:** Would it make sense to use `median()` with `withColumn()`? Why or why not? If you are unsure, try it out.</font>

In [ ]:
df_spaces.select(F.median('Site_Area_Ha').alias('Area_Median')).show()

+-------------------+
|        Area_Median|
+-------------------+
|0.04439999908208847|
+-------------------+



<font color='blue'>**Task:** According to the [data definition document](https://nottmcitycouncil.maps.arcgis.com/sharing/rest/content/items/c35ec1017beb414c82f8ec509149b5a6/data) for this dataset, the `Whole_Part` column shows whether the site is a `'Whole Site'` or is `'Part of larger site.`. When calculating the total area of open spaces, "*'Part of larger site' features should be excluded*". **Compute the total area of open spaces in the city, and name the resulting DataFrame column `Total_Area`.**</font>

<font color='blue'>You will need to keep only the relevant rows of the dataset and find the appropriate function for adding up values in a DataFrame column.</font>

In [ ]:
df_spaces.filter(df_spaces.Whole_Part == 'Whole Site').select(F.sum('Site_Area_Ha').alias('Total_Area')).show()

+------------------+
|        Total_Area|
+------------------+
|2229.9436952816322|
+------------------+



<font color='blue'>**Task:** Find the function that will calculate percentiles of a column. Inside a single `select()`, calculate the 5th and 95th percentiles of the `Site_Area_Ha` function, naming the columns `'5%'` and `'95%'` respectively. Assign the resulting dataframe to a variable `percentile_df` - we will use this below.</font>

In [ ]:
percentile_df = df_spaces.select(F.percentile("Site_Area_Ha", 0.05).alias('5%'),F.percentile("Site_Area_Ha", 0.95).alias('95%'))
percentile_df.show()

+--------------------+------------------+
|                  5%|               95%|
+--------------------+------------------+
|0.003040000051259995|3.7388599395751925|
+--------------------+------------------+



<font color='blue'>**Task:** Filter `df_spaces` to keep only rows where `Site_Area_Ha` is larger than the 5th percentile. You should do this entirely programmatically, i.e. rather than hardcoding the value for the filter you retrieve it instead from `percentile_df`. Assign the resulting DataFrame back to the `df_spaces` variable. You may want to `count()` the DataFrame before and after to check that you have had the desired effect!</font>

In [ ]:
print(f'Size of df_spaces before: {df_spaces.count()}')

pc_5 = percentile_df.collect()[0]['5%']
print(f'5th percentile is {pc_5}')

df_spaces = df_spaces.filter(df_spaces.Site_Area_Ha > pc_5)
print(f'Size of df_spaces after: {df_spaces.count()}')

Size of df_spaces before: 2449
5th percentile is 0.003040000051259995
Size of df_spaces after: 2326


<font color='blue'>**Task:** Get the rows for spaces with an area less than 1 hectare as a new DataFrame called `df_small`. Add a new column `Site_Area_m2` to `df_small` that gives the site area in metres squared. Hint: 1 hectare is 10,000m<sup>2</sup>. </font>

In [ ]:
df_small = df_spaces.filter(df_spaces.Site_Area_Ha < 1.0)
df_small = df_small.withColumn('Site_Area_m2', df_small.Site_Area_Ha * 10000)
df_small.show()

+-------+--------------------+-------------+--------------------+--------------------+-------------------+------------+--------------------+------------+
|Site_ID|           Site_Name|    Hierarchy|         Site_Access|          Site_Owner|         Whole_Part|Site_Area_Ha|                Ward|Site_Area_m2|
+-------+--------------------+-------------+--------------------+--------------------+-------------------+------------+--------------------+------------+
|     62| Victoria Embankment|  Destination|Freely accessible...|Nottingham City C...|Part of larger site|      0.2454|             Meadows|      2454.0|
|   2414|Highfields Sports...|         NULL|No access informa...|                NULL|Part of larger site|      0.9731|Lenton & Wollaton...|      9731.0|
|     20|Forest Recreation...|         City|Freely accessible...|Nottingham City C...|Part of larger site|      0.9192|Hyson Green & Arb...|      9192.0|
|   2481|Springhead Woodla...|         NULL|      Not Accessible|           

### Tidying up the data

As with most datasets, this one is imperfect. Let's focus on the `Site_Owner` column and see if we can make the values more consistent.

First of all, we'll check how many null values `Site_Owner` contains.

In [ ]:
df_spaces.filter(F.isnull('Site_Owner')).count()

155

<font color='blue'>**Task:** 'True' nulls may not be the only occurrences of missing information. Display the distinct values from the `Site_Owner` column **as a list** to see if there are any other problems.</font>

In [ ]:
[r.Site_Owner for r in df_spaces.select('Site_Owner').distinct().collect()]

['<Null>',
 'David Wilson Homes',
 'NHS',
 'Southwell Diocese',
 'Church',
 'Notts Trent Uni',
 'Notts Wildlife Trust',
 'Spirita Housing',
 'Various',
 'Not NCC - Unknown',
 'Nottingham Prison',
 'Private',
 'Unknown',
 'Management company',
 'Metropolitan Housing',
 'Nottingham Trent Uni',
 'University of Nottingham',
 'Nottingham Girls High',
 'Nottingham City Council',
 'Nottingahm City Council',
 'Nottingham City Homes',
 'Nottingham High School',
 'Housing Association',
 ' ',
 'Scouts',
 'Nottingham City council',
 None]

From this, we can see there are a few things in the `Site_Owner` column that could do with tidying up:

* **Missing values:**
  * Genuine missing values - these showed as NULL when `.show()` was used, and as `None` in the list above.
  * Missing values where a value has been entered - these show as `'<Null>'` and `' '` (these would have been missed if you just relied on `isnull()`).
* **Unknown `Site_Owner`:** These show as `'Unknown'` and `'Not NCC - Unknown'` in the list above.
* **Inconsistencies:** If you look at the list above you will see three instances of 'Nottingham City Council'. One instance has 'Nottingham' misspelt, and one instance has a lower case 'c' on Council.

In order to do this, we want to make a new DataFrame where some values are replaced according to some conditions. Below is an example of conditionally replacing values in the `Hierarchy` column for you to refer to. The expression is quite lengthy, so I have put it into a separate variable!

In [ ]:
print('BEFORE:')
df_spaces.show(5)

expr = F.when(df_spaces['Hierarchy'] == 'Neighbourhood', 'Neighbourhood Only').otherwise(df_spaces['Hierarchy'])
df2 = df_spaces.withColumn('Hierarchy', expr)
print('AFTER:')
df2.show(5)

BEFORE:
+-------+--------------------+-------------+--------------------+--------------------+-------------------+------------+------------+
|Site_ID|           Site_Name|    Hierarchy|         Site_Access|          Site_Owner|         Whole_Part|Site_Area_Ha|        Ward|
+-------+--------------------+-------------+--------------------+--------------------+-------------------+------------+------------+
|     62| Victoria Embankment|  Destination|Freely accessible...|Nottingham City C...|Part of larger site|      0.2454|     Meadows|
|   2401|Elliott Durham Sc...|         NULL|    No public access|                NULL|Part of larger site|       1.995|   Mapperley|
|     93|Hempshill Lane Re...|Neighbourhood|Freely accessible...|                NULL|Part of larger site|      1.9917|     Bulwell|
|    266|Birchover Recreat...|Neighbourhood|Freely accessible...|                NULL|Part of larger site|      1.9537|  Bilborough|
|   2391|Clifton Village C...|         NULL|No access informa

<font color='blue'>**Task:** Tidy up the `Site_Owner` column according to the following. Assign the resulting DataFrame to a `df_new` variable.</font>

* <font color='blue'>Missing values should all be recorded as an actual NULL (hint: although `show()` presents these as NULL, behind the scenes these are `None`).</font>
* <font color='blue'>Unknown site owners should always be recorded as '`Unknown`'.</font>
* <font color='blue'>All instances of Nottingham City Council should be correctly spelt with consistent capitalisation.</font>

In [ ]:
df_new = df_spaces

expr = F.when(df_new['Site_Owner'] == 'Nottingham City council', 'Nottingham City Council').otherwise(df_new['Site_Owner'])
df_new = df_new.withColumn('Site_Owner', expr)

expr = F.when(df_new['Site_Owner'] == 'Nottingahm City Council', 'Nottingham City Council').otherwise(df_new['Site_Owner'])
df_new = df_new.withColumn('Site_Owner', expr)

expr = F.when(df_new['Site_Owner'] == 'Not NCC - Unknown', 'Unknown').otherwise(df_new['Site_Owner'])
df_new = df_new.withColumn('Site_Owner', expr)

expr = F.when(df_new['Site_Owner'] == '<Null>', None).otherwise(df_new['Site_Owner'])
df_new = df_new.withColumn('Site_Owner', expr)

expr = F.when(df_new['Site_Owner'] == ' ', None).otherwise(df_new['Site_Owner'])
df_new = df_new.withColumn('Site_Owner', expr)

You can run the below cell to sanity check your resulting `df_new`.

In [ ]:
print(df_new.filter(F.isnull('Site_Owner')).count()) # this number should have gone up
[r.Site_Owner for r in df_new.select('Site_Owner').distinct().collect()] # the erroneous entries should no longer exist

169


['David Wilson Homes',
 'NHS',
 'Southwell Diocese',
 'Church',
 'Notts Trent Uni',
 'Notts Wildlife Trust',
 'Spirita Housing',
 'Various',
 'Nottingham Prison',
 'Private',
 'Unknown',
 'Management company',
 'Metropolitan Housing',
 'Nottingham Trent Uni',
 'University of Nottingham',
 'Nottingham Girls High',
 'Nottingham City Council',
 'Nottingham City Homes',
 'Nottingham High School',
 'Housing Association',
 'Scouts',
 None]

## Note: Pandas API on Spark

PySpark does provide a Pandas API ([docs](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html)), allowing you to use pandas operations on PySpark DataFrames. There is not 100% compatibility between Pandas API on Spark and pandas. For the purposes of this module, we will **not** be using Pandas API on Spark; instead, we will interact with the PySpark DataFrame API - as we have done throughout this notebook!

If you do want to use Pandas API on Spark, you can find the best practice guidelines [here](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/best_practices.html#best-practices).

PySpark also provides the `DataFrame.to_pandas()` function. If you use `DataFrame.to_pandas()`, you will bring the entire DataFrame back to the driver node as a pandas DataFrame. This means the DataFrame is no longer distributed. This should only be done if you are certain the DataFrame will fit into memory. Furthermore, doing this without good reason will result in lost marks in the assessment of this module, as we should be aiming to minimise data movement wherever possible. We will provide more information on this in the coming weeks!

## Stop the underlying SparkContext

It is good practice to stop the SparkContext when you are done!

In [ ]:
spark.stop()